# TESS FFI cutout benchmarking

This notebook intends to compare the performance of the TESSCut API against consuming the AWS S3 cube cut files directly using the AstroCut package.

In [1]:
from tess_locator import TessCoord, TessCoordList
from random import randint

In [2]:
CUTOUT_SIZE = 3

In [3]:
def get_random_coordinates(n=10, sector=None, camera=None, ccd=None) -> TessCoordList:
    """Returns a list of n random TESS pixel positions."""
    return TessCoordList(
            [TessCoord(sector=sector if sector else randint(1, 39),
                       camera=camera if camera else randint(1, 4),
                       ccd=ccd if ccd else randint(1, 4),
                       column=randint(100, 2000),
                       row=randint(100, 2000))
             for idx in range(n)])

In [4]:
from astrocut import CutoutFactory

def run_astrocut_s3(crd):
    """Create a cutout using the S3-powered version of astrocut.
    
    Parameters
    ----------
    crd : TessCoord
        TESS pixel position to cut out.
    
    Returns
    -------
    target_pixel_file : str
        Local filename of the extracted Target Pixel File.
    """
    # Where is the cube file located on S3?
    cube_file = f"s3://stpubdata/tess/public/mast/tess-s{crd.sector:04d}-{crd.camera}-{crd.ccd}-cube.fits"
    # Name of the output file
    target_pixel_file = f"{hash(str(crd))}.fits"
    # Create and return the cutout
    CutoutFactory().cube_cut(cube_file,
                             coordinates=crd.to_skycoord(),
                             cutout_size=CUTOUT_SIZE,
                             target_pixel_file=target_pixel_file)
    return target_pixel_file

In [5]:
def run_tesscut(crd):
    return CUTOUT_SIZE

In [6]:
# use `multiprocess` instead of `multiprocessing` because
# the former better supports interactive notebooks
from multiprocess import Pool

def run_benchmark(func, n_cutouts=1, cutout_size=10, sector=None, processes=None):
    # Default number of processes to the number of cutouts
    if not processes:
        processes = n_cutouts
        
    # Generate random positions to cut out
    crdlist = get_random_coordinates(n=n_cutouts, sector=sector)

    # Hack: use a global constant to pass cutout size as an argument to `func`
    global CUTOUT_SIZE
    CUTOUT_SIZE = cutout_size
    
    # Run the target function
    with Pool(processes) as p:
        result = p.map(func, crdlist)

    return result

In [7]:
%%time
result = run_benchmark(run_tesscut, n_cutouts=1, cutout_size=3, sector=10)

CPU times: user 6.15 ms, sys: 4.21 ms, total: 10.4 ms
Wall time: 9.92 ms


In [9]:
%%time
result = run_benchmark(run_astrocut_s3, n_cutouts=1, cutout_size=3, sector=10)

RuntimeError: There is no current event loop in thread 'Thread-11'.